In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVR
from scipy.stats import uniform, randint

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
df = pd.read_csv('Данные для построения модели, задача - Регрессия для SI.csv')
df

,FpDensityMorgan1,VSA_EState1,PEOE_VSA7,MinEStateIndex,Chi0v,Chi0n,fr_urea,BCUT2D_MWHI,VSA_EState8,Kappa3,...,MinPartialCharge,fr_Imine,MaxAbsEStateIndex,fr_methoxy,BCUT2D_MWLOW,PEOE_VSA6,MinAbsPartialCharge,"CC50, mM",FpDensityMorgan3,Target Feature
0,0.642857,0.000000,74.032366,0.387225,19.534409,19.534409,0,14.822266,16.981087,2.868737,...,-0.293526,2,5.094096,0,9.700470,54.384066,0.038844,175.482382,1.321429,1.449093
1,0.607143,0.000000,97.951860,0.533868,19.794682,19.794682,0,14.975110,17.670565,3.027177,...,-0.313407,0,3.961417,0,9.689226,54.384066,0.012887,5.402819,1.285714,0.845098
2,0.562500,2.517630,74.032366,0.543231,23.689110,23.689110,0,15.353938,18.287216,3.470070,...,-0.325573,0,2.627117,0,9.681293,41.542423,0.094802,161.142320,1.156250,-0.142668
3,0.620690,0.000000,74.032366,0.390603,20.241516,20.241516,0,14.821216,17.012013,3.263848,...,-0.293526,2,5.097360,0,9.700497,60.804888,0.038844,107.855654,1.310345,1.800960
4,0.600000,0.000000,103.003916,0.270476,22.617677,22.617677,0,14.831112,14.692318,3.345855,...,-0.257239,2,5.150510,0,9.700386,65.807891,0.062897,139.270991,1.257143,0.113943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1.133333,10.503509,68.114460,-0.476142,18.825334,18.825334,0,16.586886,8.824371,2.219273,...,-0.468587,0,12.934891,1,9.344314,38.841158,0.317890,34.999650,2.533333,0.052701
997,1.085714,10.086396,68.114460,-0.699355,21.810933,21.810933,0,16.586914,10.378794,3.147136,...,-0.467493,0,13.635345,1,9.343622,45.764895,0.327562,33.999415,2.457143,0.026322
998,1.157895,10.305031,79.620167,-0.650790,24.449891,23.633394,0,32.166365,8.774745,3.822745,...,-0.467485,0,13.991690,1,9.343613,45.764895,0.327887,33.999458,2.552632,0.040112
999,0.756757,20.885832,56.278648,-1.408652,23.380977,23.380977,0,16.540061,7.488627,4.164473,...,-0.468755,0,13.830180,4,9.364015,27.192033,0.312509,32.999644,1.864865,0.013373


In [3]:
X, y = df.drop('Target Feature', axis=1), df['Target Feature']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Target Feature', axis=1), df['Target Feature'], test_size=0.3, random_state=42)

#### Контрольной метрика для задачи регрессии будем считать R^2 score. Она будет записана в атрибут assessment_criterion BestRegressionModel класса

In [5]:
class BestRegressionModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели регресии
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'R^2',
            'value': 0.5
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LinearRegression(),
                'best_model': None,
                'best_params': None,
                'params': {},
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestRegressor(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': ElasticNet(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'alpha': [0.01, 0.1, 1, 10],
                    'l1_ratio': [0.2, 0.5, 0.8]
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVR(),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            self.models[model]['metrics']['MSE'] = mean_squared_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['MAE'] = mean_absolute_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['R^2'] = r2_score(y, self.models[model]['data']['y_pred'])
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"MSE: {self.models[model]['metrics']['MSE']:.4f}")
            print(f"MAE: {self.models[model]['metrics']['MAE']:.4f}")
            print(f"R^2: {self.models[model]['metrics']['R^2']:.4f}")
            print()
            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return 10 ** result

#### Подбор гипперпараметров для моделей

In [6]:
best_regression_model = BestRegressionModel(X, y)
best_regression_model.fit()


Подбор гипперпараметров для модели LinearRegression() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели LinearRegression(): {}

Подбор гипперпараметров для модели RandomForestRegressor(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestRegressor(random_state=42): {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 67}

Подбор гипперпараметров для модели ElasticNet(random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели ElasticNet(random_state=42): {'alpha': 0.01, 'l1_ratio': 0.5}

Подбор гипперпараметров для модели SVR() методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию парамет

#### Поиск лучшей модели

In [7]:
best_regression_model.get_best_model()

Обучаем модель LinearRegression() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LinearRegression():
MSE: 0.5071
MAE: 0.5376
R^2: 0.1517

Обучаем модель RandomForestRegressor(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestRegressor(random_state=42):
MSE: 0.3690
MAE: 0.4649
R^2: 0.3827

Обучаем модель ElasticNet(random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели ElasticNet(random_state=42):
MSE: 0.5109
MAE: 0.5442
R^2: 0.1454

Обучаем модель SVR() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандарт

#### Формирование предсказания лучшей моделью

In [8]:
best_regression_model.predict(X_test)

Модель для формирования прогноза не сформирована, используйте метод get_best_model


Так как мы не смогли найти модель, которая была бы лучше, чем заданное значение метрикик, изменим значение R^2 score на 0.3

In [9]:
best_regression_model.assessment_criterion['value'] = 0.3

Попробуем снова подобрать модель

In [11]:
best_regression_model.get_best_model()

Обучаем модель LinearRegression() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LinearRegression():
MSE: 0.5071
MAE: 0.5376
R^2: 0.1517

Обучаем модель RandomForestRegressor(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestRegressor(random_state=42):
MSE: 0.3690
MAE: 0.4649
R^2: 0.3827

Обучаем модель ElasticNet(random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели ElasticNet(random_state=42):
MSE: 0.5109
MAE: 0.5442
R^2: 0.1454

Обучаем модель SVR() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандарт

In [12]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели RandomForestRegressor(max_depth=10, n_estimators=67, random_state=42)
Предсказание сформировано


array([3.85600044e+00, 3.30361885e+00, 4.50714495e+00, 1.36631281e+00,
       2.69724097e+00, 4.59125582e+00, 3.05127813e+00, 3.63897874e+00,
       1.34306725e+02, 2.77580727e+00, 7.18265428e+01, 1.25022061e+00,
       4.35508080e+00, 1.63340580e+01, 2.73642361e+00, 1.34869265e+00,
       6.95192627e+00, 6.35254087e+01, 4.66038298e+00, 5.65748766e+00,
       4.73351642e+00, 7.57449168e+01, 1.84261919e+01, 3.92195817e+00,
       3.13544889e+00, 4.88645848e+00, 2.12299634e+00, 4.17422455e+00,
       2.44733276e+00, 2.59729043e+02, 6.50470206e+00, 4.33396278e+00,
       3.48365302e+00, 2.78843438e+00, 1.55601699e+01, 4.50431385e+00,
       3.33254598e+00, 4.53715517e+00, 1.84817226e+01, 2.39288361e+01,
       3.69097685e+00, 2.92601332e+01, 5.00124865e+00, 9.88718752e+00,
       9.14671168e+00, 3.85703555e+00, 3.81036402e+00, 4.34007363e+00,
       3.95695874e+00, 3.33000268e+00, 4.36407431e+00, 2.68784047e+01,
       2.68926023e+00, 3.13454887e+00, 2.36284887e+00, 1.83560934e+01,
      

#### Для улучшения показателей моделей, необходимо еще раз произвести подбор параметров, путем увеличиения выборки (топ признаков), для каждого метода отбора